# CONTRE: Example

This is an abstract and easy example to show the usage of the CONTRE Continuum Reweighting.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Generation of Files

In this abstract example, we have the following components and variables:  
- `componentA`, 
- `componentB` (only on resonance).
- `variable1` (badly simulated for `componentA`)
- `variable2`.

In the following cell these samples are generated and stored in `example_input`.

In [ ]:
%run -i generate_data.py

### Histogram of the example data:  
You can also look at `"variable2"`.

In [ ]:
variable = "variable1"
fig, ax = plt.subplots(1,2, figsize=[12.8, 4.8])

# on resonance histogram
count, edges = np.histogram(
    data[variable], bins=30, range=(0,1))

bin_width=edges[1] - edges[0]
bin_mids = edges[:-1]+bin_width
ax[0].plot(
    bin_mids, count, color="black", marker='.', ls="",
    label="data") 

w = size_data/size_mc
ax[0].hist(
    [componentA[variable], componentB[variable]],
    bins=30, range=(0,1), stacked=True,
    weights=[[w]*len(componentA), [w]*len(componentB)],
    label=["componentA", "componentB"])

ax[0].set_title("On resonance")
ax[0].legend()

# off resonance histogram
count, edges = np.histogram(
    data_offres[variable], bins=30, range=(0,1))
ax[1].plot(
    bin_mids, count, color="black", marker='.', ls="")

ax[1].hist(
    componentA_offres[variable], bins=30, range=(0,1),
    weights=[w]*len(componentA_offres))

ax[1].set_title("Off resonance")

plt.show()

## Setting the parameters

To start the training you need to set the parameters by writing them to a json file.
An example `example_parameters.json` file is created in the following cells.

## Starting the training
The Training is implemented with `b2luigi`. Set a result path and run the process command as shown below.

```python
test(a) = c
len(a)
```

## Finding and using the output